In [1]:
import pandas as pd
import numpy as np
import pymongo as pymongo
import json
from pandas.io.json import json_normalize

In [2]:
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

In [3]:
client = pymongo.MongoClient("idenmon.zapto.org", 888)
db = client.prodLaboratorio
pruebas = db.joinedLabApp


In [4]:
dfGlobalOrig = pd.DataFrame(list(pruebas.find()))
lGlobal=list(pruebas.find())

In [5]:
dfGlobal=pd.DataFrame(lGlobal)
dfGlobal=dfGlobalOrig
#dfGlobal.head()

In [6]:
g=globals()
proteinas =['CFP10',  'ESAT6',  'RV1681',  'P24']
for proteina in proteinas:
    g['{0}Counts'.format(proteina)] = dfGlobal[proteina+'_Comp'].groupby(dfGlobal['appVersion']).value_counts()

In [7]:
versiones = ['0.1.0', '0.2.1', '0.3.1'] #automatizarlo para extraer las versiones del DataFrame
mezclaVersionesProteinas = [(ver,prot) for ver in versiones for prot in proteinas]
d={}
#for ver, prot in mezclaVersionesProteinas:
for ver in versiones:
    v = ver.replace('.','')
    for prot in proteinas:
        g['{0}'.format(prot)]={}
        resProt = g['{0}Counts'.format(prot)].loc[ver]
        g['sens_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0) + resProt.get('FN',0)))
        #g['prec_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['specif_{0}'.format(prot)]= safe_div(resProt.get('TN',0), (resProt.get('TN',0)+resProt.get('FP',0)))
        g['posPV_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['negPV_{0}'.format(prot)]= safe_div(resProt.get('TN',0),(resProt.get('TN',0)+resProt.get('FN',0)))
        g['{0}'.format(prot)].update({'Sensitivity': g['sens_{0}'.format(prot)] })
        #g['{0}'.format(prot)].update({'Prescition': g['prec_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Specificity': g['specif_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Positive Predictive Value': g['posPV_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Negative Predictive Value': g['negPV_{0}'.format(prot)] })
        totalPruebas = resProt.get('TP',0)+resProt.get('FP',0)+resProt.get('TN',0)+resProt.get('FN',0)+resProt.get('IND',0)
        g['{0}'.format(prot)].update({'#TotalPruebas': totalPruebas})
        g['{0}'.format(prot)].update({'#TP': resProt.get('TP',0) })
        g['{0}'.format(prot)].update({'#FP': resProt.get('FP',0) })
        g['{0}'.format(prot)].update({'#TN': resProt.get('TN',0) })
        g['{0}'.format(prot)].update({'#FN': resProt.get('FN',0) })
        g['{0}'.format(prot)].update({'#Indet': resProt.get('IND',0)})
        g['{0}'.format(prot)].update({'#Invalidos': resProt.get('INVALID',0)})
        d.update({prot: g['{0}'.format(prot)]})
    g['df{0}'.format(v)] = pd.DataFrame.from_dict(d, orient='Index')

In [8]:
df021T = df021.T
df010T = df010.T
df031T = df031.T
dfComp = df010T.join(df021T,lsuffix='__v0.1.0', rsuffix='__v0.2.1')
dfComp = dfComp.join(df031T, rsuffix='__v0.3.1')
dfComp

,CFP10__v0.1.0,ESAT6__v0.1.0,P24__v0.1.0,RV1681__v0.1.0,CFP10__v0.2.1,ESAT6__v0.2.1,P24__v0.2.1,RV1681__v0.2.1,CFP10,ESAT6,P24,RV1681
Sensitivity,0.526316,0.645161,0.782609,0.828125,0.875000,0.875000,0.875000,0.8125,0.0,0.0,0.0,0.0
Specificity,0.936508,0.916256,1.000000,0.945000,1.000000,1.000000,1.000000,1.0000,1.0,1.0,1.0,1.0
Positive Predictive Value,0.769231,0.701754,1.000000,0.828125,1.000000,1.000000,1.000000,1.0000,0.0,0.0,0.0,0.0
Negative Predictive Value,0.830986,0.894231,0.137931,0.945000,0.333333,0.333333,0.333333,0.2500,1.0,1.0,1.0,1.0
#TotalPruebas,316.000000,316.000000,316.000000,316.000000,17.000000,17.000000,17.000000,17.0000,45.0,45.0,45.0,45.0
#TP,40.000000,40.000000,180.000000,53.000000,14.000000,14.000000,14.000000,13.0000,0.0,0.0,0.0,0.0
#FP,12.000000,17.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
#TN,177.000000,186.000000,8.000000,189.000000,1.000000,1.000000,1.000000,1.0000,45.0,45.0,45.0,45.0
#FN,36.000000,22.000000,50.000000,11.000000,2.000000,2.000000,2.000000,3.0000,0.0,0.0,0.0,0.0
#Indet,51.000000,51.000000,78.000000,52.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0


In [9]:
CFP10Counts

appVersion  CFP10_Comp
0.1.0       TN            177
            IND            51
                           44
            TP             40
            FN             36
            FP             12
            INVALID        11
0.2.0       FN              4
            FP              2
0.2.1       TP             14
                            8
            FN              2
            INVALID         2
            TN              1
0.3.1       TN             45
Name: CFP10_Comp, dtype: int64